# Embedding-Vektoren in Large Language Models

Das Folgende zeigt, wie man die Embedding-Vektoren erhalten und mit ihnen rechnen kann. Um die nötigen Komponenten zu installieren, empfiehlt es sich, die Anaconda-Python-Distribution zu benutzen. Dann kann man auf der Konsole des Betriebssystems die folgenden Befehle geben:

conda create -n lmm-demo python=3.8

conda activate lmm-demo

conda install -c conda-forge transformers

conda install pytorch torchvision torchaudio cpuonly -c pytorch


Das folgende zeigt, wie man die Vektoren bekommt und mit ihnen rechnet. Die Ähnlichkeiten sind bei Googles Bert generell recht hoch, aber die Unterschiede sind schon so, wie man es erwartet.

In [1]:
import torch
from transformers import AutoTokenizer, AutoModel
import math

# Laden des M odells - hier Bert (beim erstn mal dauert das lange) 
model_name = "bert-base-uncased" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_word_embedding(word):
    # Erzeugen der Token
    inputs = tokenizer(word, return_tensors='pt')    
    with torch.no_grad():
        outputs = model(**inputs)
    # Extraktion der mbeddings
    embeddings = outputs.last_hidden_state
    if len(embeddings)>1: print(f"Es gab {len(embeddings)} Tokens. Nur das erste wird genutzt.")
    e=embeddings[0][0]  # Embedding des ersten Tokens
    return e.tolist() # Rückgabe als Python-Liste

embedMan = get_word_embedding("man")
embedWoman = get_word_embedding("woman")
embedKing = get_word_embedding("king")
embedQueen = get_word_embedding("queen")
embed7 = get_word_embedding("seven")

# Vektoroperationen mit Listen als Vektoren
def vadd(a,b): return([a[i]+b[i] for i in range(min(len(a),len(b)))])
def vsub(a,b): return([a[i]-b[i] for i in range(min(len(a),len(b)))])
def vsmul(s,a): return([s*a[i] for i in range(len(a))])
def skalarProdukt(a,b): return(sum([a[i]*b[i] for i in range(min(len(a),len(b)))]))
def vnorm(a): return(math.sqrt(skalarProdukt(a,a)))   
def vnormed(a): return(vsmul(1.0/vnorm(a),a))    
def cosineSim(a,b): return(skalarProdukt(vnormed(a),vnormed(b)))
    
print(f"Infos zum Embedding von 'queen': Dimension {len(embedQueen)}, Länge des Vektors {vnorm(embedQueen)}")

# Scalar product (dot product)
print(f"Skalarprodukt von 'man' und 'woman' : {skalarProdukt(embedMan,embedWoman)}")
print(f"Skalarprodukt von 'man' und 'queen' : {skalarProdukt(embedMan,embedQueen)}")
print(f"Skalarprodukt von 'man' und '7' : {skalarProdukt(embedMan,embed7)}")
print(f"Cosinus-Ähnlichkeit von 'man' und 'woman' : {cosineSim(embedMan,embedWoman)}")
print(f"Cosinus-Ähnlichkeit von 'man' und 'queen' : {cosineSim(embedMan,embedQueen)}")
print(f"Cosinus-Ähnlichkeit von 'man' und '7' : {cosineSim(embedMan,embed7)}")

Infos zum Embedding von 'queen': Dimension 768, Länge des Vektors 14.485814105400754
Skalarprodukt von 'man' und 'woman' : 205.74431258703873
Skalarprodukt von 'man' und 'queen' : 202.07533714101365
Skalarprodukt von 'man' und '7' : 195.95884762206447
Cosinus-Ähnlichkeit von 'man' und 'woman' : 0.9754535546998419
Cosinus-Ähnlichkeit von 'man' und 'queen' : 0.9619835011071921
Cosinus-Ähnlichkeit von 'man' und '7' : 0.9384714423875936


Mit den Vektoren kann man rechnen: König-Mann+Frau sollte in der Nähe von Königin liegen. Die Berechnung ist etwas geringer als erwartet, aber immerhin

In [2]:
queenBerechnet=vadd(vsub(embedKing,embedMan),embedWoman)
print(f"Cosinus-Ähnlichkeit von 'queen' und der berechneten Königin : {cosineSim(embedQueen,queenBerechnet)}")

Cosinus-Ähnlichkeit von 'queen' und der berechneten Königin : 0.9572869600125741
